In [ ]:
from __future__ import print_function

import pandas as pd
import numpy as np
import ipywidgets

from ipyleaflet import *

In [ ]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors

def n_colors(n, colormap=mpl.cm.Blues):
    data = np.linspace(0.0,1.0,n)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

def data_to_colors(data, colormap=mpl.cm.plasma):
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(mpl.colors.Normalize()(data))]
    return c

In [ ]:
m1 = Map(center=[37.7749, -122.34580993652344], zoom=12, layout=Layout(height='600px'))

In [ ]:
survey_data = pd.read_csv('./../analysis_data/scores_by_zipcodes.csv', header=None)

In [ ]:
colors = data_to_colors(survey_data.ix[:, 1], colormap=mpl.cm.Greens)

In [ ]:
zips = [str(int(i)) for i in survey_data.ix[:, 0]]

In [ ]:
cols = {}
for i in range(len(zips)):
    cols[zips[i]] = colors[i]

In [ ]:
import json
with open('./../mapdata/sf_zipcodes.geojson') as f:
    data = json.load(f)

In [ ]:
for feature in data['features']:
    feature['properties']['style'] = {
        'color': cols[feature['id']],
        'weight': 1,
        'fillColor': cols[feature['id']],
        'fillOpacity': 0.75,
    }

In [ ]:
g = GeoJSON(data=data)
m1 += g

In [ ]:
m1

In [ ]:
m = Map(center=[37.7749, -122.34580993652344], zoom=12, layout=Layout(height='600px'))

In [ ]:
tract_data = pd.read_csv('./../data/CensusData/sfo data/tract_data_normalized.csv')

In [ ]:
colors = data_to_colors(tract_data['pct_bachelors'], colormap=mpl.cm.Reds)

In [ ]:
tracts = [str(int(i)) for i in tract_data.ix[:, 0].values]

In [ ]:
cols = {}
for i in range(len(tracts)):
    cols[tracts[i][4:]] = colors[i]

In [ ]:
import json
with open('./../mapdata/sf_tracts.geojson') as f:
    data = json.load(f)

In [ ]:
for feature in data['features']:
    try:
        style_col = cols[feature['properties']['tractce10']]
    except KeyError:
        style_col = 'Grey'
    feature['properties']['style'] = {
        'color': style_col,
        'weight': 1,
        'fillColor': style_col,
        'fillOpacity':0.75,
    }

In [ ]:
g = GeoJSON(data=data)
m += g

In [ ]:
from ipywidgets import Dropdown, VBox

In [ ]:
from IPython.display import display

In [ ]:
x = Dropdown(options=list(tract_data.columns)[2:], value='pct_bachelors')

In [ ]:
def on_changed(name, value):
    #if len(m.layers) > 0:
    #    m.remove_layer(m.layers[-1])
    colors = data_to_colors(tract_data[x.value], colormap=mpl.cm.Reds)
    cols = {}
    for i in range(len(tracts)):
        cols[tracts[i][4:]] = colors[i]
    for feature in data['features']:
        try:
            style_col = cols[feature['properties']['tractce10']]
        except KeyError:
            style_col = 'grey'
        feature['properties']['style'] = {
            'color': style_col,
            'weight': 1,
            'fillColor': style_col,
            'fillOpacity':0.75,
        }
    g = GeoJSON(data=data)
    m.layers = [m.layers[0], g]
    
x.on_trait_change(on_changed, 'value')

In [ ]:
display(VBox([x]))

In [ ]:
m